# Задание
Вы работаете аналитиком команды монетизации. Тестируете новую систему цен на очень небольшую аудиторию – профессионалы в небольшом городе.

Аудитория достаточно пассивная, поэтому в ней редко "прокрашиваются" фичи. Каждая потенциальная фича на вес золота.

Менеджеры имеют большие надежды на эту фичу.

Основная метрика ARPPU. Даже такое небольшое повышение позволит начать зарабатывать больше.

**Вам нужно**

- провести первичный анализ и найти наблюдаемый эффект между группами
- проанализировать базовый A/B-тест и понять есть ли стат. значимое изменение в post данных после внедрения новых цен
- провести CUPED t-test
- рассчитать размер выборки для CUPED t-test

**Описание данных**

Данные содержат поюзерную пару ARPPU до теста (pre_ARPPU) и ARPU после тесте (post_ARPU)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
df = pd.read_csv("cuped_homework.csv")

In [3]:
df.head()

,post_ARPPU,pre_ARPPU,group
0,660.0,595.0,A
1,540.0,621.0,A
2,863.0,782.0,A
3,431.0,567.0,A
4,434.0,473.0,A


### Задание 1. Первичный анализ (1/2)
Найдите наблюдаемую абсолютную разницу между средними у Б и А групп.

Введите значение в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [4]:
group_a = df[df["group"] == "A"]
group_b = df[df["group"] == "B"]

In [5]:
mean_diff = group_b["post_ARPPU"].mean() - group_a["post_ARPPU"].mean()

round(mean_diff, 3)

np.float64(5.356)

### Задание 1. Первичный анализ (2/2)
Укажите наблюдаемый `effect size Cohen D`. Для расчетов используйте абсолютную разницу и стандартное отклонение в pre данных в группе А.

Введите значение в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [6]:
cohen_d = mean_diff / group_a["pre_ARPPU"].std()

round(cohen_d, 3)

np.float64(0.054)

### Задание 2. Простой t-test
Найдите p-value, полученное при проведении простого t-test на post данных.

Введите значение в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [7]:
_, pval = stats.ttest_ind(group_a["post_ARPPU"], group_b["post_ARPPU"])

In [8]:
round(pval, 3)

np.float64(0.074)

### Задание 3. CUPED t-test (1/2)
Найдите долю пустых значений в pre данных.

Введите значение в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [9]:
round(df["pre_ARPPU"].isnull().sum() / len(df), 3)

np.float64(0.133)

### Задание 3. CUPED t-test (2/2)
Найдите p-value в проведенном CUPED t-test. Предварительно заполните пустые значения средней на pre периоде.

Введите значение в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [10]:
df["pre_ARPPU"].fillna(df["pre_ARPPU"].mean(), inplace=True)

/var/folders/m6/gnxqn1753_g808tzdcqqmdq80000gn/T/ipykernel_35951/505537658.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["pre_ARPPU"].fillna(df["pre_ARPPU"].mean(), inplace=True)


In [11]:
df["pre_ARPPU"].isnull().sum()

np.int64(0)

In [12]:
def calculate_theta_for_test(control_pre, control_post, test_pre, test_post):
    theta = (
        np.cov(control_post, control_pre)[0, 1]
        + np.cov(test_post, test_pre)[0, 1]
    ) / (np.var(control_pre) + np.var(test_pre))
    return theta


def get_basic_ttest(group_A, group_B):
    """Проверяет гипотезу о равенстве средних для обычного среднего.
    return - t_stat, p_value"""

    t_stat, p_value = stats.ttest_ind(group_A, group_B)
    inference = {"t_stat": t_stat, "p_value": p_value}
    return inference


def get_cuped_ttest(control_pre, control_post, test_pre, test_post):
    """Проверяет гипотезу о равенстве средних CUPED вариант
    return - t_stat, p_value"""

    theta = calculate_theta_for_test(
        control_pre, control_post, test_pre, test_post
    )

    control_cuped = control_post - theta * control_pre
    test_cuped = test_post - theta * test_pre

    inference = get_basic_ttest(control_cuped, test_cuped)

    return inference

In [13]:
group_a = df[df["group"] == "A"]
group_b = df[df["group"] == "B"]

In [14]:
cuped_test = get_cuped_ttest(
    group_a["pre_ARPPU"],
    group_a["post_ARPPU"],
    group_b["pre_ARPPU"],
    group_b["post_ARPPU"],
)

In [15]:
round(cuped_test["p_value"], 3)

np.float64(0.032)